In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import Input

In [2]:
def create_image_paths(images_ids):
    images_paths = []
    img = plt.imread('C:/Users/mahmo/Desktop/Leaf-Classification/input/images/1.jpg')
    source_folder = 'C:/Users/mahmo/Desktop/Leaf-Classification/input/images/'
    for id in images_ids:
        img_path = source_folder + str(id) + '.jpg'
        images_paths.append(img_path)
    return images_paths

In [3]:
def draw_some_images(images_paths):
    fig , axis = plt.subplots(2,4)
    fig.set_figheight(10)
    fig.set_figwidth(20)
    # load images
    images = []
    for path in images_paths:
        images.append(plt.imread(path))
    i , j = 0 , 0
    for img in images:
        axis[i,j].imshow(img,cmap='gray')
        j = j + 1
        if j == 4:
            i = i + 1
            j = 0

In [4]:
def read_data(file_name):
    # read data from csv file in input folder
    cur_dir = os.getcwd()
    file_path = 'C:/Users/mahmo/Desktop/Leaf-Classification/input/' + file_name
    data = pd.read_csv(os.path.join(cur_dir,file_path))
    return data

In [5]:
def clean_data(train_data, test_data):
    X_train = train_data.drop(columns=['species','id'])
    y_train = train_data[['species']]
    # rename test variable and drop id column from the test dat
    X_test = test_data.drop(columns=['id'])
    # determine unique classes in the dataset
    y_train_unique = pd.unique(y_train['species'])
    # length of unique classes
    y_train_unique_length = len(y_train_unique)
    # view correlations between columns
    correlation_matrix = X_train.corr()
    # take absolute values
    correlation_matrix_abs = correlation_matrix.abs()
    upper_matrix = correlation_matrix_abs.where(np.triu(np.ones(correlation_matrix_abs.shape),k=1).astype(np.bool_))
    # check to see if there is a column has a correlation higher than 0.9
    correlation_threshold = 0.9
    col_with_high_correlation = [col for col in upper_matrix.columns if np.any(upper_matrix[col] > correlation_threshold)]
    # drop shape columns from X_train and also X_test
    X_train = X_train.drop(columns=col_with_high_correlation)
    X_test = X_test.drop(columns=col_with_high_correlation)
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = pd.DataFrame(scaler.transform(X_train), index = X_train.index, columns = X_train.columns)
    # apply the transformation to the X_test data
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), index = X_test.index, columns = X_test.columns)
    y_train_encoded = pd.get_dummies(y_train['species'])
    return X_train_scaled, y_train_encoded, X_test_scaled, y_train_unique_length

### Custom Callback

In [6]:
class Mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}):
        if logs['accuracy'] > 0.99:
            print('\nReached required accuracy above 99% at epoch', epochs, ', so cancel training!')
            self.model.stop_training = True

### Model Architecture

In [7]:
# define the model architecture
def DenseMLP(input_shape, hidden_size, output_size, dropout_rate, dropout = False):
    if dropout:
        # (batch_size,input_shape[0],input_shape[1])
        model = Sequential([
        # input layer
        Input(shape=(input_shape[-1])),
        # one hidden layer
        Dense(units = hidden_size, activation = 'tanh'),
        # dropout layer
        Dropout(rate = dropout_rate),
        # output layer
        Dense(units = output_size, activation = 'softmax')
        ])
    else:
        model = Sequential([
        # input layer
        Input(shape=(input_shape[-1])),
        # one hidden layer
        Dense(units = hidden_size, activation = 'tanh'),
        # output layer
        Dense(units = output_size, activation = 'softmax')
        ])
    
    return model

### Training Function

In [8]:
# define the training function which accepts many hyperparameters
def training(model, X, y, batch_size, optimizer, learning_rate_scheduler, custom_call_back):
    # compile the model
    model.compile(optimizer = optimizer, loss = CategoricalCrossentropy(), metrics = ['accuracy'])
    # callbacks
    if learning_rate_scheduler is not None:
        lr_scheduler_callback = learning_rate_scheduler
        if custom_call_back is None:
            history = model.fit(x= X,y= y, batch_size= batch_size, epochs = 500, verbose= 0, 
                            callbacks= [lr_scheduler_callback], validation_split= 0.2)
        else:
            on_epoch_end_callback = Mycallback()
            # model fit
            history = model.fit(x= X,y= y, batch_size= batch_size, epochs = 500, verbose= 0, 
                                callbacks= [lr_scheduler_callback, on_epoch_end_callback], validation_split= 0.2)
    else:
        if custom_call_back is None:
            history = model.fit(x= X,y= y, batch_size= batch_size, epochs = 500, verbose= 0, validation_split= 0.2)
        else:
            on_epoch_end_callback = Mycallback()
            # model fit
            history = model.fit(x= X,y= y, batch_size= batch_size, 
                                epochs = 500, verbose= 0, callbacks= [on_epoch_end_callback], validation_split= 0.2)
    return history

### Evaluation Function

In [9]:
# define the evaluation function which loads the trained model and evaluate its performance on train/test set
def evaluation(model, X, y, batch_size):
    results = model.evaluate(x= X, y= y, batch_size = batch_size, verbose= 0)
    return results[0], results[1]

### Training Curves Function

In [10]:
def train_curves(history, special_title, special_title_var):
    if special_title_var is None:
        title_loss = 'Model loss per epoch ' + special_title
        title_accuracy = 'Model accuracy per epoch ' + special_title
    else:    
        title_loss = 'Model loss per epoch ' + special_title + ' ' + str(special_title_var)
        title_accuracy = 'Model accuracy per epoch ' + special_title + ' ' + str(special_title_var)
    fig , axis = plt.subplots(nrows=1, ncols=2)
    # dimensions of figure
    fig.set_figheight(6)
    fig.set_figwidth(14)
    # loss
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    # accuracy
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    epoch = np.arange(150)
    # loss curve
    axis[0].plot(loss,label='Train')
    axis[0].plot(val_loss,label='Validation')
    axis[0].set_xlabel('epoch')
    axis[0].set_ylabel('loss')
    axis[0].set_title(title_loss)
    axis[0].legend()
    # accuracy curve
    axis[1].plot(accuracy, label='Train')
    axis[1].plot(val_accuracy, label='Validation')
    axis[1].set_xlabel('epoch')
    axis[1].set_ylabel('accuracy')
    axis[1].set_title(title_accuracy)
    axis[1].legend()